In [ ]:
#default_exp data.core

In [ ]:
# make your Google drive accessible 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai2_library/course-v4/'

# navigate to the notebooks directory for dl2
import os
os.chdir(base_dir)

Mounted at /content/gdrive


In [ ]:
!pwd
# cd to base_dir if above os.chdir does not work using below command
# %cd "/content/gdrive/My Drive/fastai2_library/course-v4/"

/content/gdrive/My Drive/fastai2_library/course-v4


In [ ]:
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install fastai==2.0.9

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 358kB 8.3MB/s 
     |████████████████████████████████| 40kB 5.8MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
%cd nbs

/content/gdrive/My Drive/fastai2_library/course-v4/nbs


In [ ]:
#export
from fastai.torch_basics import *
from fastai.data.load import *

In [ ]:
#hide
#from nbdev.showdoc import *

# Data core

> Core functionality for gathering data

The classes here provide functionality for applying a list of transforms to a set of items (`TfmdLists`, `Datasets`) or a `DataLoader` (`TfmdDl`) as well as the base class used to gather the data for model training: `DataLoaders`.

## TfmdDL -

In [ ]:
#export
@typedispatch
def show_batch(x, y, samples, ctxs=None, max_n=9, **kwargs):
    if ctxs is None: ctxs = Inf.nones
    if hasattr(samples[0], 'show'):
        ctxs = [s.show(ctx=c, **kwargs) for s,c,_ in zip(samples,ctxs,range(max_n))]
    else:
        for i in range_of(samples[0]):
            ctxs = [b.show(ctx=c, **kwargs) for b,c,_ in zip(samples.itemgot(i),ctxs,range(max_n))]
    return ctxs

`show_batch` is a type-dispatched function that is responsible for showing decoded `samples`. `x` and `y` are the input and the target in the batch to be shown, and are passed along to dispatch on their types. There is a different implementation of `show_batch` if `x` is a `TensorImage` or a `TensorText` for instance (see vision.core or text.data for more details). `ctxs` can be passed but the function is responsible to create them if necessary. `kwargs` depend on the specific implementation.

In [ ]:
#export
@typedispatch
def show_results(x, y, samples, outs, ctxs=None, max_n=9, **kwargs):
    if ctxs is None: ctxs = Inf.nones
    for i in range(len(samples[0])):
        ctxs = [b.show(ctx=c, **kwargs) for b,c,_ in zip(samples.itemgot(i),ctxs,range(max_n))]
    for i in range(len(outs[0])):
        ctxs = [b.show(ctx=c, **kwargs) for b,c,_ in zip(outs.itemgot(i),ctxs,range(max_n))]
    return ctxs

`show_results` is a type-dispatched function that is responsible for showing decoded `samples` and their corresponding `outs`. Like in `show_batch`, `x` and `y` are the input and the target in the batch to be shown, and are passed along to dispatch on their types. `ctxs` can be passed but the function is responsible to create them if necessary. `kwargs` depend on the specific implementation.

In [ ]:
#export
_all_ = ["show_batch", "show_results"]

In [ ]:
#export
_batch_tfms = ('after_item','before_batch','after_batch')

In [ ]:
#export
@log_args(but_as=DataLoader.__init__)
@delegates()
class TfmdDL(DataLoader):
    "Transformed `DataLoader`"
    # Due to @delegates decorator the kwargs below passed to the superclass DataLoader in super().__init__
    # will be correctly documented and will TAB complete.
    def __init__(self, dataset, bs=64, shuffle=False, num_workers=None, verbose=False, do_setup=True, **kwargs):
        if num_workers is None: num_workers = min(16, defaults.cpus)
         # The specific list of callbacks in _batch_tfms list are turned into Transform Pipelines
        # so after you grab a single item in the dataset there will be a pipeline
        # After you get all the items but before you collate them, there will be the
        # before_batch callback and its transform pipeline,  
        # After things have been turned into a batch there will be after_batch callback
        # transform pipeline
        for nm in _batch_tfms: kwargs[nm] = Pipeline(kwargs.get(nm,None))
        super().__init__(dataset, bs=bs, shuffle=shuffle, num_workers=num_workers, **kwargs)
        # Calls do_setup if each of the pipline transforms has a setup.
        if do_setup:
            for nm in _batch_tfms:
                pv(f"Setting up {nm}: {kwargs[nm]}", verbose)
                kwargs[nm].setup(self)

    def _one_pass(self):
        # do_batch on do_item to get b and
        # find type of its after batching b.
        b = self.do_batch([self.do_item(0)])
        if self.device is not None: b = to_device(b, self.device)
        its = self.after_batch(b)
        self._n_inp = 1 if not isinstance(its, (list,tuple)) or len(its)==1 else len(its)-1
        self._types = explode_types(its)

    def _retain_dl(self,b):
        # A function to figure out types during decode
        # Also done in new below
        # do _one_pass see above
        if not getattr(self, '_types', None): self._one_pass()
        # retain_types function knows how to ensure that things
        # keep their types
        return retain_types(b, typs=self._types)

    @delegates(DataLoader.new)
    def new(self, dataset=None, cls=None, **kwargs):
        res = super().new(dataset, cls, do_setup=False, **kwargs)
        if not hasattr(self, '_n_inp') or not hasattr(self, '_types'):
            try:
                self._one_pass()
                res._n_inp,res._types = self._n_inp,self._types
            except: print("Could not do one pass in your dataloader, there is something wrong in it")
        else: res._n_inp,res._types = self._n_inp,self._types
        return res

    def before_iter(self):
        super().before_iter()
        split_idx = getattr(self.dataset, 'split_idx', None)
        for nm in _batch_tfms:
            f = getattr(self,nm)
            if isinstance(f,Pipeline): f.split_idx=split_idx
    # when we execute decode we can put back the type using the _retail_dl
    def decode(self, b): return self.before_batch.decode(to_cpu(self.after_batch.decode(self._retain_dl(b))))
    def decode_batch(self, b, max_n=9, full=True): return self._decode_batch(self.decode(b), max_n, full)

    def _decode_batch(self, b, max_n=9, full=True):
        f = self.after_item.decode
        f = compose(f, partial(getattr(self.dataset,'decode',noop), full = full))
        return L(batch_to_samples(b, max_n=max_n)).map(f)

    # Used in show_batch but does the decode to recall/put back the correct
    # types and call the decode bits of all of our different pipelines and allow
    # to show. If we can show whole batch at once for things like tabular
    # most datasets we have to display each part of the tuple separately eg image
    # and label, which is handled here.
    def _pre_show_batch(self, b, max_n=9):
        "Decode `b` to be ready for `show_batch`"
        b = self.decode(b)
        if hasattr(b, 'show'): return b,None,None
        its = self._decode_batch(b, max_n, full=False)
        if not is_listy(b): b,its = [b],L((o,) for o in its)
        return detuplify(b[:self.n_inp]),detuplify(b[self.n_inp:]),its
    
    # When show_batch is executed it calls _pre_show_batch
    def show_batch(self, b=None, max_n=9, ctxs=None, show=True, unique=False, **kwargs):
        if unique:
            old_get_idxs = self.get_idxs
            self.get_idxs = lambda: Inf.zeros
        if b is None: b = self.one_batch()
        if not show: return self._pre_show_batch(b, max_n=max_n)
        show_batch(*self._pre_show_batch(b, max_n=max_n), ctxs=ctxs, max_n=max_n, **kwargs)
        if unique: self.get_idxs = old_get_idxs

    def show_results(self, b, out, max_n=9, ctxs=None, show=True, **kwargs):
        x,y,its = self.show_batch(b, max_n=max_n, show=False)
        b_out = type(b)(b[:self.n_inp] + (tuple(out) if is_listy(out) else (out,)))
        x1,y1,outs = self.show_batch(b_out, max_n=max_n, show=False)
        res = (x,x1,None,None) if its is None else (x, y, its, outs.itemgot(slice(self.n_inp,None)))
        if not show: return res
        show_results(*res, ctxs=ctxs, max_n=max_n, **kwargs)

    @property
    def n_inp(self):
        if hasattr(self.dataset, 'n_inp'): return self.dataset.n_inp
        if not hasattr(self, '_n_inp'): self._one_pass()
        return self._n_inp

    def to(self, device):
        self.device = device
        for tfm in self.after_batch.fs:
            for a in L(getattr(tfm, 'parameters', None)): setattr(tfm, a, getattr(tfm, a).to(device))
        return self

A `TfmdDL` is a `DataLoader` that creates `Pipeline` from a list of `Transform`s for the callbacks `after_item`, `before_batch` and `after_batch`. As a result, it can decode or show a processed `batch`.

In [ ]:
#export
add_docs(TfmdDL,
         decode="Decode `b` using `tfms`",
         decode_batch="Decode `b` entirely",
         new="Create a new version of self with a few changed attributes",
         show_batch="Show `b` (defaults to `one_batch`), a list of lists of pipeline outputs (i.e. output of a `DataLoader`)",
         show_results="Show each item of `b` and `out`",
         before_iter="override",
         to="Put self and its transforms state on `device`")

In [ ]:
class _Category(int, ShowTitle): pass

In [ ]:
#Test retain type
class NegTfm(Transform):
    def encodes(self, x): return torch.neg(x)
    def decodes(self, x): return torch.neg(x)

In [ ]:
#Test retain type using None to allow for setting of type to Tensor instead
# of TensorImage
class NegTfmNone(Transform):
    def encodes(self, x)->None: return torch.neg(x)
    def decodes(self, x): return torch.neg(x)

In [ ]:
# Only coz torch.neg does not care about length of x, we can put NegTfm() in
# after_batch callback NOT OW. IF we had added CUDA it could have been on GPU
# Since NO CUDA this is still on CPU - even though it is after_batch
tdl = TfmdDL([(TensorImage([1]),)] * 4, after_batch=NegTfm(), bs=4, num_workers=4)
b = tdl.one_batch()
print(b)
print()
print(b[0])
print()
# Check that after going through Transform Type remains same as input to dataloader
# This is so even though torch.neg generates a Tensor 
# Now TensorImage is subclass of Tensor and the right subclass type is provided to 
# b[0] ie it is of type TensorImage and NOT type Tensor after going through 
# NegTfm().

# SO TfmdDL and Dataset check that after going through Transforms/Pipelines
# that the type DOES NOT CHANGE. IF IT DOES CHANGE - it will change it BACK 
# TO WHAT IT USED TO BE. 

# IF YOU DO NOT WANT THIS BEHAVIOR - OPT OUT by putting 
# RETURN TYPE of None
#
test_eq(type(b[0]), TensorImage)
b = (tensor([1.,1.,1.,1.]),)
print(b[0][0])
# Key thing to note here that due to TypeDispatch the TensorImage type is 
# attached to the data that comes from the data loader  

# This is handy for Inference for in production your predictions can have
# proper types so you can call normal methods and Transforms on them

# Also for applying stuff to the Test Set 
# 
test_eq(type(tdl.decode_batch(b)[0][0]), TensorImage)

(TensorImage([[-1],
        [-1],
        [-1],
        [-1]]),)

TensorImage([[-1],
        [-1],
        [-1],
        [-1]])

tensor(1.)


In [ ]:
# Check that after going through Transform Type DOES NOT remain same as input 
# to dataloader because NegTfmNone uses None for return allowing opting out
# of Transform's inherent ability to retain types going through Transform

# So unlike for NegTfm, for NegTfmNone b is a tensor  
# 

# IF YOU DO NOT WANT THIS BEHAVIOR - OPT OUT by putting 
# RETURN TYPE of None
#
tdl = TfmdDL([(TensorImage([1]),)] * 4, after_batch=NegTfmNone(), bs=4, num_workers=4)
b = tdl.one_batch()
print(b)
print()
print(b[0])
print()

test_eq(type(b[0]), Tensor)
b = (tensor([1.,1.,1.,1.]),)
print(b[0][0])

(tensor([[-1],
        [-1],
        [-1],
        [-1]]),)

tensor([[-1],
        [-1],
        [-1],
        [-1]])

tensor(1.)


In [ ]:
class A(Transform): 
    def encodes(self, x): return x 
    def decodes(self, x): return TitledInt(x) 

# Note that there is no decode for this function
# so on decode it just returns its input
@Transform
def f(x)->None: return fastuple((x,x))

start = torch.arange(50)
test_eq_type(f(2), fastuple((2,2)))

In [ ]:
'''
Signature: tdl.one_batch()
Source:   
    def one_batch(self):
        if self.n is not None and len(self)==0: raise ValueError(f'This DataLoader does not contain any batches')
        with self.fake_l.no_multiproc(): res = first(self)
        if hasattr(self, 'it'): delattr(self, 'it')
        return res
File:      /usr/local/lib/python3.6/dist-packages/fastai2/data/load.py
Type:      method
'''
tdl.one_batch??

In [ ]:
'''
Signature: first(x)
Source:   
def first(x):
    "First element of `x`, or None if missing"
    try: return next(iter(x))
    except StopIteration: return None
File:      /usr/local/lib/python3.6/dist-packages/fastcore/utils.py
'''
first??

In [ ]:
a = A()
tdl = TfmdDL(start, after_item=lambda x: (a(x), f(x)), bs=4)
# note that one_batch() is just next(iter(self)) of a tfmdDL
# alternately first(x) as shown above
x,y = tdl.one_batch()
print(x)
print()
print(y)
print()
test_eq(type(y), fastuple)

s = tdl.decode_batch((x,y))
print(s)
print()
print(s[0][1])
test_eq(type(s[0][1]), fastuple)
#skr tests
print(s[0][0])
# Q - why is below a Tensor - would expect it to be TitledInt see A above 
# Ans: NO - since default behavior of Transforms is to retain passed in type 
# and the input to decodes is a Tensor the output will also be a Tensor even if
# decodes method return type is TitledInt - ofcourse unless behavior is opted
# OUT OF by providing None as output type of decodes
test_eq(type(s[0][0]), Tensor)

tensor([0, 1, 2, 3])

(tensor([0, 1, 2, 3]), tensor([0, 1, 2, 3]))

(#4) [(tensor(0), (tensor(0), tensor(0))),(tensor(1), (tensor(1), tensor(1))),(tensor(2), (tensor(2), tensor(2))),(tensor(3), (tensor(3), tensor(3)))]

(tensor(0), tensor(0))
tensor(0)


In [ ]:
tdl = TfmdDL(torch.arange(0,50), after_item=A(), after_batch=NegTfm(), bs=4)
test_eq(tdl.dataset[0], start[0])
# see cells below for explanation
test_eq(len(tdl), (50-1)//4+1)
test_eq(tdl.bs, 4)
test_stdout(tdl.show_batch, '0\n1\n2\n3')
test_stdout(partial(tdl.show_batch, unique=True), '0\n0\n0\n0')

In [ ]:
(50 - 1) // 4 

12

In [ ]:
# get the // 4 and + 1 but why 50 - 1??
(50 - 1) // 4 + 1

13

In [ ]:
batch_val = tdl.show_batch()
print(batch_val)
batch_val_unique = partial(tdl.show_batch, unique=True)()
print(batch_val_unique)

0
1
2
3
None
0
0
0
0
None


In [ ]:
'''
Signature: detuplify(x)
Source:   
def detuplify(x):
    "If `x` is a tuple with one thing, extract it"
    return None if len(x)==0 else x[0] if len(x)==1 and getattr(x, 'ndim', 1)==1 else x
File:      /usr/local/lib/python3.6/dist-packages/fastcore/utils.py
'''
detuplify??

In [ ]:
x = ((1),)
value1 = detuplify(x)
print(value1)
x = (())
value2 = detuplify(x)
print(value2)
x = ((1),)
# so this creates a dim attribute and assigns value of that to 1
# if that is true it returns x[0] else returns x 
# so that getattr part is to distinguish between the two cases of 
# x = (1,) in which case return x vs x = ((1),) in which case return x[0]
print(getattr(x, 'ndim', 1)) 
print()
x = (1,)
value3 = detuplify(x)
print(value3)

1
None
1

1


In [ ]:
class B(Transform):
    parameters = 'a'
    def __init__(self): self.a = torch.tensor(0.)
    def encodes(self, x): x
    
tdl = TfmdDL([(TensorImage([1]),)] * 4, after_batch=B(), bs=4)
test_eq(tdl.after_batch.fs[0].a.device, torch.device('cpu'))
tdl.to(default_device())
test_eq(tdl.after_batch.fs[0].a.device, default_device())

### Methods

In [ ]:
show_doc(TfmdDL.one_batch)

<h4 id="DataLoader.one_batch" class="doc_header"><code>DataLoader.one_batch</code><a href="https://github.com/fastai/fastai/tree/master/fastai/data/load.py#L130" class="source_link" style="float:right">[source]</a></h4>

> <code>DataLoader.one_batch</code>()



In [ ]:
tfm = NegTfm()
tdl = TfmdDL(start, after_batch=tfm, bs=4)

In [ ]:
b = tdl.one_batch()
test_eq(tensor([0,-1,-2,-3]), b)

In [ ]:
show_doc(TfmdDL.decode)

<h4 id="TfmdDL.decode" class="doc_header"><code>TfmdDL.decode</code><a href="__main__.py#L44" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdDL.decode</code>(**`b`**)

Decode `b` using `tfms`

In [ ]:
# Note that this is just negTfm() decode which neg its input so gets back original
test_eq(tdl.decode(b), tensor(0,1,2,3))

In [ ]:
show_doc(TfmdDL.decode_batch)

<h4 id="TfmdDL.decode_batch" class="doc_header"><code>TfmdDL.decode_batch</code><a href="__main__.py#L45" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdDL.decode_batch</code>(**`b`**, **`max_n`**=*`9`*, **`full`**=*`True`*)

Decode `b` entirely

In [ ]:
test_eq(tdl.decode_batch(b), [0,1,2,3])

In [ ]:
show_doc(TfmdDL.show_batch)

<h4 id="TfmdDL.show_batch" class="doc_header"><code>TfmdDL.show_batch</code><a href="__main__.py#L60" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdDL.show_batch</code>(**`b`**=*`None`*, **`max_n`**=*`9`*, **`ctxs`**=*`None`*, **`show`**=*`True`*, **`unique`**=*`False`*, **\*\*`kwargs`**)

Show `b` (defaults to `one_batch`), a list of lists of pipeline outputs (i.e. output of a [`DataLoader`](data.load#DataLoader))

In [ ]:
show_doc(TfmdDL.to)

<h4 id="TfmdDL.to" class="doc_header"><code>TfmdDL.to</code><a href="__main__.py#L83" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdDL.to</code>(**`device`**)

Put self and its transforms state on `device`

# What is covered at beginning of WT 3 Video

In [ ]:
''
Signature: to_device(b, device=None)
Source:   
def to_device(b, device=None):
    "Recursively put `b` on `device`."
    if defaults.use_cuda==False: device='cpu'
    elif device is None: device=default_device()
    def _inner(o): return o.to(device, non_blocking=True) if isinstance(o,Tensor) else o.to_device(device) if hasattr(o, "to_device") else o
    # appply applies _inner recursively to any list, tuple, dict etc ie to all elems of those 
    return apply(_inner, b)
File:      /usr/local/lib/python3.6/dist-packages/fastai2/torch_core.py
'''
to_device??

In [ ]:
'''
Signature: default_device(use_cuda=-1)
Source:   
def default_device(use_cuda=-1):
    "Return or set default device; `use_cuda`: None - CUDA if available; True - error if not availabe; False - CPU"
    if use_cuda != -1: defaults.use_cuda=use_cuda
    use = defaults.use_cuda or (torch.cuda.is_available() and defaults.use_cuda is None)
    assert torch.cuda.is_available() or not use
    return torch.device(torch.cuda.current_device()) if use else torch.device('cpu')
File:      /usr/local/lib/python3.6/dist-packages/fastai2/torch_core.py
'''
default_device??

In [ ]:
# Can use torch.cuda.set_device() to set set the device to be used 
default_device()

device(type='cpu')

In [ ]:
# Note below apply is fastai apply - rather than changing something it returns the result of applying
# and it is making every effort to keep all the types consistent as fastai does everywhere so if your listy thing
# is actually some subclass it will make sure that it keeps that subclass all the way through.
# 

In [ ]:
'''
Signature: apply(func, x, *args, **kwargs)
Source:   
def apply(func, x, *args, **kwargs):
    "Apply `func` recursively to `x`, passing on args"
    if is_listy(x): return type(x)([apply(func, o, *args, **kwargs) for o in x])
    if isinstance(x,dict):  return {k: apply(func, v, *args, **kwargs) for k,v in x.items()}
    res = func(x, *args, **kwargs)
    return res if x is None else retain_type(res, x)
File:      /usr/local/lib/python3.6/dist-packages/fastai2/torch_core.py
'''
apply??

In [ ]:
'''
Signature: docs(cls)
Source:   
def docs(cls):
    "Decorator version of `add_docs`, using `_docs` dict"
    add_docs(cls, **cls._docs)
    return cls
File:      /usr/local/lib/python3.6/dist-packages/fastcore/foundation.py
'''
docs??

In [ ]:
'''
Signature: add_docs(cls, cls_doc=None, **docs)
Source:   
def add_docs(cls, cls_doc=None, **docs):
    "Copy values from `docs` to `cls` docstrings, and confirm all public methods are documented"
    if cls_doc is not None: cls.__doc__ = cls_doc
    for k,v in docs.items():
        f = getattr(cls,k)
        if hasattr(f,'__func__'): f = f.__func__ # required for class methods
        f.__doc__ = v
    # List of public callables without docstring
    nodoc = [c for n,c in vars(cls).items() if callable(c)
             and not n.startswith('_') and c.__doc__ is None]
    assert not nodoc, f"Missing docs: {nodoc}"
    assert cls.__doc__ is not None, f"Missing class docs: {cls}"
'''
add_docs??

In [ ]:
GetAttr??

## DataLoaders -

Note that there is DataLoader class (note singular) which is different and is in 02_data.load notebook which also inherits from GetAttr - that DataLoader has the one_batch method.

In [ ]:
'''
Init signature: GetAttr(*args, **kwargs)
Source:        
class GetAttr:
    "Inherit from this to have all attr accesses in `self._xtra` passed down to `self.default`"
    _default='default'
    def _component_attr_filter(self,k):
        if k.startswith('__') or k in ('_xtra',self._default): return False
        xtra = getattr(self,'_xtra',None)
        return xtra is None or k in xtra
    def _dir(self): return [k for k in dir(getattr(self,self._default)) if self._component_attr_filter(k)]
    def __getattr__(self,k):
        if self._component_attr_filter(k):
            attr = getattr(self,self._default,None)
            if attr is not None: return getattr(attr,k)
        raise AttributeError(k)
    def __dir__(self): return custom_dir(self,self._dir())
#     def __getstate__(self): return self.__dict__
    def __setstate__(self,data): self.__dict__.update(data)
File:           /usr/local/lib/python3.6/dist-packages/fastcore/foundation.py

In [ ]:
# export
@docs
class DataLoaders(GetAttr):
    "Basic wrapper around several `DataLoader`s."
    _default='train'
    def __init__(self, *loaders, path='.', device=None):
        self.loaders,self.path = list(loaders),Path(path)
        if device is not None or hasattr(loaders[0],'to'): self.device = device

    def __getitem__(self, i): return self.loaders[i]
    def new_empty(self):
        loaders = [dl.new(dl.dataset.new_empty()) for dl in self.loaders]
        return type(self)(*loaders, path=self.path, device=self.device)

    def _set(i, self, v): self.loaders[i] = v
    train   ,valid    = add_props(lambda i,x: x[i], _set)
    train_ds,valid_ds = add_props(lambda i,x: x[i].dataset)

    @property
    def device(self): return self._device

    @device.setter
    def device(self, d):
        for dl in self.loaders: dl.to(d)
        self._device = d

    def to(self, device):
        self.device = device
        return self

    def cuda(self): return self.to(device=default_device())
    def cpu(self):  return self.to(device=torch.device('cpu'))

    @classmethod
    def from_dsets(cls, *ds, path='.',  bs=64, device=None, dl_type=TfmdDL, **kwargs):
        default = (True,) + (False,) * (len(ds)-1)
        defaults = {'shuffle': default, 'drop_last': default}
        for nm in _batch_tfms:
            if nm in kwargs: kwargs[nm] = Pipeline(kwargs[nm])
        kwargs = merge(defaults, {k: tuplify(v, match=ds) for k,v in kwargs.items()})
        kwargs = [{k: v[i] for k,v in kwargs.items()} for i in range_of(ds)]
        return cls(*[dl_type(d, bs=bs, **k) for d,k in zip(ds, kwargs)], path=path, device=device)

    @classmethod
    def from_dblock(cls, dblock, source, path='.',  bs=64, val_bs=None, shuffle_train=True, device=None, **kwargs):
        return dblock.dataloaders(source, path=path, bs=bs, val_bs=val_bs, shuffle_train=shuffle_train, device=device, **kwargs)

    _docs=dict(__getitem__="Retrieve `DataLoader` at `i` (`0` is training, `1` is validation)",
               train="Training `DataLoader`",
               valid="Validation `DataLoader`",
               train_ds="Training `Dataset`",
               valid_ds="Validation `Dataset`",
               to="Use `device`",
               cuda="Use the gpu if available",
               cpu="Use the cpu",
               new_empty="Create a new empty version of `self` with the same transforms",
               from_dblock="Create a dataloaders from a given `dblock`")

In [ ]:
tdl.data

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [ ]:
b = tdl.one_batch()
b

tensor([ 0, -1, -2, -3])

In [ ]:
'''
Signature: first(x)
Source:   
def first(x):
    "First element of `x`, or None if missing"
    try: return next(iter(x))
    except StopIteration: return None
File:      /usr/local/lib/python3.6/dist-packages/fastcore/utils.py
'''
first??

In [ ]:
# DataLoaders just wrapper around multiple loaders
dls = DataLoaders(tdl,tdl)
# x is one batch of dls train dataloader 
x = dls.train.one_batch()
# x2 is first (one_batch) of tdl - which ought to be same 
# as dls is just a wrapper and dls.train is just first dataloader
x2 = first(tdl)
# Hence x and x2 are equal
test_eq(x,x2)
# Below test shows that dls.one_batch() and ds.train.one_batch() are the same
# This is because _default in dataloaders is 'train' (see above)
x2 = dls.one_batch()
test_eq(x,x2)

In [ ]:
'''
Signature: add_props(f, g=None, n=2)
Source:   
def add_props(f, g=None, n=2):
    "Create properties passing each of `range(n)` to f"
    if g is None: return (property(partial(f,i)) for i in range(n))
    return (property(partial(f,i), partial(g,i)) for i in range(n))
File:      /usr/local/lib/python3.6/dist-packages/fastcore/utils.py
'''
add_props??

In [ ]:
#hide
#test assignment works
dls.train = dls.train.new(bs=4)

### Methods

In [ ]:
show_doc(DataLoaders.__getitem__)

<h4 id="DataLoaders.__getitem__" class="doc_header"><code>DataLoaders.__getitem__</code><a href="__main__.py#L10" class="source_link" style="float:right">[source]</a></h4>

> <code>DataLoaders.__getitem__</code>(**`i`**)

Retrieve [`DataLoader`](data.load#DataLoader) at `i` (`0` is training, `1` is validation)

In [ ]:
x2 = dls[0].one_batch()
test_eq(x,x2)

In [ ]:
show_doc(DataLoaders.train, name="DataLoaders.train")

<h4 id="DataLoaders.train" class="doc_header"><code>DataLoaders.train</code><a href="__main__.py#L16" class="source_link" style="float:right">[source]</a></h4>

Training [`DataLoader`](data.load#DataLoader)

In [ ]:
show_doc(DataLoaders.valid, name="DataLoaders.valid")

<h4 id="DataLoaders.valid" class="doc_header"><code>DataLoaders.valid</code><a href="__main__.py#L16" class="source_link" style="float:right">[source]</a></h4>

Validation [`DataLoader`](data.load#DataLoader)

In [ ]:
show_doc(DataLoaders.train_ds, name="DataLoaders.train_ds")

<h4 id="DataLoaders.train_ds" class="doc_header"><code>DataLoaders.train_ds</code><a href="__main__.py#L17" class="source_link" style="float:right">[source]</a></h4>

Training `Dataset`

In [ ]:
show_doc(DataLoaders.valid_ds, name="DataLoaders.valid_ds")

<h4 id="DataLoaders.valid_ds" class="doc_header"><code>DataLoaders.valid_ds</code><a href="__main__.py#L17" class="source_link" style="float:right">[source]</a></h4>

Validation `Dataset`

## TfmdLists -

In [ ]:
#export
class FilteredBase:
    "Base class for lists with subsets"
    _dl_type,_dbunch_type = TfmdDL,DataLoaders
    def __init__(self, *args, dl_type=None, **kwargs):
        if dl_type is not None: self._dl_type = dl_type
        self.dataloaders = delegates(self._dl_type.__init__)(self.dataloaders)
        super().__init__(*args, **kwargs)

    @property
    def n_subsets(self): return len(self.splits)
    def _new(self, items, **kwargs): return super()._new(items, splits=self.splits, **kwargs)
    def subset(self): raise NotImplemented

    def dataloaders(self, bs=64, val_bs=None, shuffle_train=True, n=None, path='.', dl_type=None, dl_kwargs=None,
                    device=None, **kwargs):
        if device is None: device=default_device()
        if dl_kwargs is None: dl_kwargs = [{}] * self.n_subsets
        if dl_type is None: dl_type = self._dl_type
        drop_last = kwargs.pop('drop_last', shuffle_train)
        dl = dl_type(self.subset(0), bs=bs, shuffle=shuffle_train, drop_last=drop_last, n=n, device=device,
                     **merge(kwargs, dl_kwargs[0]))
        dls = [dl] + [dl.new(self.subset(i), bs=(bs if val_bs is None else val_bs), shuffle=False, drop_last=False,
                             n=None, **dl_kwargs[i]) for i in range(1, self.n_subsets)]
        return self._dbunch_type(*dls, path=path, device=device)

FilteredBase.train,FilteredBase.valid = add_props(lambda i,x: x.subset(i))

TfmdLists: Pass in a list of items and a list of transforms, it creates a pipeline with those transforms, and due to inheriting from L and super().__init__ items items are in a L.

self.splits becomes multiple L's if splits is not None

if tfms are already TfmdLists then tfms = tfms.tfms

if tfms are already Pipelines set their do_setup to False (do_setup indicates whether Pipeline.setup method should be called during initalization).

if not make tfms Pipelines and set split_idx to passed in split_idx

This is how (see example in tests below) you get to access TfmdLists as Ls, so we can subscript into it, and apply the Pipeline to it.

So you begin to see some nice PyTorch dataset like behavior with TfmdLists. But keep in mind that a Dataset has an x and a y typically. Here we only have one thing in TfmdLists.

Aside: L is designed to let us create some more interesting type of collections. When you call \_\_getitem\_\_ in L, it actually calls self._get(idx) if idx is an iterator o.w. it calls L(self._get(idx), use_list=None). As long as i is not an iterator or slice, it just returns the i'th element. If an iterator or slice you can do interesting things like use this to access a pandas dataframe or an array or an iterator - see details in _get method of L.

In [ ]:
#export
class TfmdLists(FilteredBase, L, GetAttr):
    "A `Pipeline` of `tfms` applied to a collection of `items`"
    _default='tfms'
    def __init__(self, items, tfms, use_list=None, do_setup=True, split_idx=None, train_setup=True,
                 splits=None, types=None, verbose=False, dl_type=None):
        super().__init__(items, use_list=use_list)
        if dl_type is not None: self._dl_type = dl_type
        self.splits = L([slice(None),[]] if splits is None else splits).map(mask2idxs)
        if isinstance(tfms,TfmdLists): tfms = tfms.tfms
        if isinstance(tfms,Pipeline): do_setup=False
        self.tfms = Pipeline(tfms, split_idx=split_idx)
        store_attr('types,split_idx')
        if do_setup:
            pv(f"Setting up {self.tfms}", verbose)
            self.setup(train_setup=train_setup)

    def _new(self, items, split_idx=None, **kwargs):
        split_idx = ifnone(split_idx,self.split_idx)
        return super()._new(items, tfms=self.tfms, do_setup=False, types=self.types, split_idx=split_idx, **kwargs)
    def subset(self, i): return self._new(self._get(self.splits[i]), split_idx=i)
    def _after_item(self, o): return self.tfms(o)
    def __repr__(self): return f"{self.__class__.__name__}: {self.items}\ntfms - {self.tfms.fs}"
    def __iter__(self): return (self[i] for i in range(len(self)))
    def show(self, o, **kwargs): return self.tfms.show(o, **kwargs)
    def decode(self, o, **kwargs): return self.tfms.decode(o, **kwargs)
    def __call__(self, o, **kwargs): return self.tfms.__call__(o, **kwargs)
    def overlapping_splits(self): return L(Counter(self.splits.concat()).values()).filter(gt(1))
    def new_empty(self): return self._new([])

    def setup(self, train_setup=True):
        self.tfms.setup(self, train_setup)
        if len(self) != 0:
            x = super().__getitem__(0) if self.splits is None else super().__getitem__(self.splits[0])[0]
            self.types = []
            for f in self.tfms.fs:
                self.types.append(getattr(f, 'input_types', type(x)))
                x = f(x)
            self.types.append(type(x))
        types = L(t if is_listy(t) else [t] for t in self.types).concat().unique()
        self.pretty_types = '\n'.join([f'  - {t}' for t in types])

    def infer_idx(self, x):
        # TODO: check if we really need this, or can simplify
        idx = 0
        for t in self.types:
            if isinstance(x, t): break
            idx += 1
        types = L(t if is_listy(t) else [t] for t in self.types).concat().unique()
        pretty_types = '\n'.join([f'  - {t}' for t in types])
        assert idx < len(self.types), f"Expected an input of type in \n{pretty_types}\n but got {type(x)}"
        return idx

    def infer(self, x):
        return compose_tfms(x, tfms=self.tfms.fs[self.infer_idx(x):], split_idx=self.split_idx)

    def __getitem__(self, idx):
        res = super().__getitem__(idx)
        if self._after_item is None: return res
        return self._after_item(res) if is_indexer(idx) else res.map(self._after_item)

In [ ]:
#export
add_docs(TfmdLists,
         setup="Transform setup with self",
         decode="From `Pipeline`",
         show="From `Pipeline`",
         overlapping_splits="All splits that are in more than one split",
         subset="New `TfmdLists` with same tfms that only includes items in `i`th split",
         infer_idx="Finds the index where `self.tfms` can be applied to `x`, depending on the type of `x`",
         infer="Apply `self.tfms` to `x` starting at the right tfm depending on the type of `x`",
         new_empty="A new version of `self` but with no items")

In [ ]:
#exports
def decode_at(o, idx):
    "Decoded item at `idx`"
    return o.decode(o[idx])

In [ ]:
#exports
def show_at(o, idx, **kwargs):
    "Show item at `idx`",
    return o.show(o[idx], **kwargs)

A `TfmdLists` combines a collection of object with a `Pipeline`. `tfms` can either be a `Pipeline` or a list of transforms, in which case, it will wrap them in a `Pipeline`. `use_list` is passed along to `L` with the `items` and `split_idx` are passed to each transform of the `Pipeline`. `do_setup` indicates if the `Pipeline.setup` method should be called during initialization.

In [ ]:
class _IntFloatTfm(Transform):
    def encodes(self, o):  return TitledInt(o)
    def decodes(self, o):  return TitledFloat(o)
int2f_tfm=_IntFloatTfm()

def _neg(o): return -o
neg_tfm = Transform(_neg, _neg)

In [ ]:
items = L([1.,2.,3.]); tfms = [neg_tfm, int2f_tfm]
tl = TfmdLists(items, tfms=tfms)
test_eq_type(tl[0], TitledInt(-1))
test_eq_type(tl[1], TitledInt(-2))
# explain next 2 lines why the diff?
test_eq_type(tl.decode(tl[2]), TitledFloat(3.))
test_stdout(lambda: show_at(tl, 2), '-3')
test_eq(tl.types, [float, float, TitledInt])
tl

TfmdLists: [1.0, 2.0, 3.0]
tfms - (#2) [_neg:
encodes: (object,object) -> _negdecodes: (object,object) -> _neg,_IntFloatTfm:
encodes: (object,object) -> encodes
decodes: (object,object) -> decodes
]

In [ ]:
# add splits to TfmdLists
splits = [[0,2],[1]]
tl = TfmdLists(items, tfms=tfms, splits=splits)
test_eq(tl.n_subsets, 2)
test_eq(tl.train, tl.subset(0))
test_eq(tl.valid, tl.subset(1))
test_eq(tl.train.items, items[splits[0]])
test_eq(tl.valid.items, items[splits[1]])
test_eq(tl.train.tfms.split_idx, 0)
test_eq(tl.valid.tfms.split_idx, 1)
test_eq(tl.train.new_empty().split_idx, 0)
test_eq(tl.valid.new_empty().split_idx, 1)
test_eq_type(tl.splits, L(splits))
assert not tl.overlapping_splits()

In [ ]:
df = pd.DataFrame(dict(a=[1,2,3],b=[2,3,4]))
tl = TfmdLists(df, lambda o: o.a+1, splits=[[0],[1,2]])
# coz o.a+1 and tl at index [1, 2] will be a+1 at index [1, 2]
# which are 2+1, 3+1 hence [3, 4]
test_eq(tl[1,2], [3,4])
# since subset 0 will be just index0 and index0 a is 1 so 
# o.a+1 = 2 hence tr[:] is 2
tr = tl.subset(0)
test_eq(tr[:], [2])
# similar logic subset 1 has 2 values [3, 4]
val = tl.subset(1)
test_eq(val[:], [3,4])

In [ ]:
print(items)
class _B(Transform):
    def __init__(self): self.m = 0
    def encodes(self, o): return o+self.m
    def decodes(self, o): return o-self.m
    def setups(self, items): 
        # print(items)
        self.m = tensor(items).float().mean().item()

# test for setup, which updates `self.m`
tl = TfmdLists(items, _B())
test_eq(tl.m, 2)

(#3) [1.0,2.0,3.0]


Here's how we can use `TfmdLists.setup` to implement a simple category list, getting labels from a mock file list:

In [ ]:
class _Cat(Transform):
    order = 1
    def encodes(self, o):    return int(self.o2i[o])
    def decodes(self, o):    return TitledStr(self.vocab[o])
    def setups(self, items): self.vocab,self.o2i = uniqueify(L(items), sort=True, bidir=True)
tcat = _Cat()

def _lbl(o): return TitledStr(o.split('_')[0])

# Check that tfms are sorted by `order` & `_lbl` is called first
fns = ['dog_0.jpg','cat_0.jpg','cat_2.jpg','cat_1.jpg','dog_1.jpg']
tl = TfmdLists(fns, [tcat,_lbl])
exp_voc = ['cat','dog']
test_eq(tcat.vocab, exp_voc)
test_eq(tl.tfms.vocab, exp_voc)
test_eq(tl.vocab, exp_voc)
test_eq(tl, (1,0,0,0,1))
test_eq([tl.decode(o) for o in tl], ('dog','cat','cat','cat','dog'))

In [ ]:
#Check only the training set is taken into account for setup
tl = TfmdLists(fns, [tcat,_lbl], splits=[[0,4], [1,2,3]])
test_eq(tcat.vocab, ['dog'])

In [ ]:
#class A(Transform): 
#    def encodes(self, x): return x 
#    def decodes(self, x): return TitledInt(x) 


In [ ]:
print(A())
print(start)
print(NegTfm())

A:
encodes: (object,object) -> encodes
decodes: (object,object) -> decodes

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])
NegTfm:
encodes: (object,object) -> encodes
decodes: (object,object) -> decodes



In [ ]:
# checking that split_idx of 1 does not work when TfmdLists has default split_idx of 0
tfm = NegTfm(split_idx=1)
tds = TfmdLists(start, A())
tdl = TfmdDL(tds, after_batch=tfm, bs=4)
x = tdl.one_batch()
# nothing happens below coz NegTfm is NOT executed
# since its split_idx of 1 is not a match
test_eq(x, torch.arange(4))
# make split idx match and now NegTfm is applied
tds.split_idx = 1
x = tdl.one_batch()
test_eq(x, -torch.arange(4))
# check again that with split_idx of - no Tfm applied
tds.split_idx = 0
x = tdl.one_batch()
test_eq(x, torch.arange(4))

In [ ]:
tds = TfmdLists(start, A())
tdl = TfmdDL(tds, after_batch=NegTfm(), bs=4)
test_eq(tdl.dataset[0], start[0])
# note that len(tdl) is # of batches which is ((50)-1)//4 + 1 = 13
test_eq(len(tdl), (len(tds)-1)//4+1)
test_eq(tdl.bs, 4)
test_stdout(tdl.show_batch, '0\n1\n2\n3')

In [ ]:
show_doc(TfmdLists.subset)

<h4 id="TfmdLists.subset" class="doc_header"><code>TfmdLists.subset</code><a href="__main__.py#L21" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdLists.subset</code>(**`i`**)

New [`TfmdLists`](/data.core#TfmdLists) with same tfms that only includes items in `i`th split

In [ ]:
show_doc(TfmdLists.infer_idx)

<h4 id="TfmdLists.infer_idx" class="doc_header"><code>TfmdLists.infer_idx</code><a href="__main__.py#L43" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdLists.infer_idx</code>(**`x`**)

Finds the index where `self.tfms` can be applied to `x`, depending on the type of `x`

In [ ]:
show_doc(TfmdLists.infer)

<h4 id="TfmdLists.infer" class="doc_header"><code>TfmdLists.infer</code><a href="__main__.py#L54" class="source_link" style="float:right">[source]</a></h4>

> <code>TfmdLists.infer</code>(**`x`**)

Apply `self.tfms` to `x` starting at the right tfm depending on the type of `x`

In [ ]:
def mult(x): return x*2
mult.order = 2

fns = ['dog_0.jpg','cat_0.jpg','cat_2.jpg','cat_1.jpg','dog_1.jpg']
tl = TfmdLists(fns, [_lbl,_Cat(),mult])

In [ ]:
# infer_idx finds index where self.tfms can be applied to x depending on type
# so type of x given is str so can apply from index 0
test_eq(tl.infer_idx('dog_45.jpg'), 0)
# Apply self.tfms starting at idx 0 implies apply _lbl get 'dog'
# Apply _Cat() get 1, then apply mult get 2.
test_eq(tl.infer('dog_45.jpg'), 2)

print(tl.tfms)
print(tl.types)
# infex_idx of 4 is 2 coz type(4) is int which is at index 2
test_eq(tl.infer_idx(4), 2)
# infer says run tfms from that index forward to end on value x which is
# just mult 4 by 2 is 8
test_eq(tl.infer(4), 8)

test_fail(lambda: tl.infer_idx(2.0))
test_fail(lambda: tl.infer(2.0))

Pipeline: _lbl -> _Cat -> mult
[<class 'str'>, <class 'fastai.torch_core.TitledStr'>, <class 'int'>, <class 'int'>]


In [ ]:
'''
Init signature: TitledStr(*args, **kwargs)
Source:        
class TitledStr(Str, ShowTitle):
    _show_args = {'label': 'text'}
    def show(self, ctx=None, **kwargs):
        "Show self"
        return show_title(str(self), ctx=ctx, **merge(self._show_args, kwargs))
File:           /usr/local/lib/python3.6/dist-packages/fastai2/torch_core.py
Type:           type
'''
TitledStr??

In [ ]:
# skr checks
print(fns)
print()
cat = _Cat()
tl1 = TfmdLists(fns, [cat])
print(tl1.types)
tl2 = TfmdLists(fns, [_lbl])
print(tl2.types)
tl3 = TfmdLists(fns, [mult]) 
# whatever type(s) go(es) in come(s) out??
# but no output types considered per cat() example above??
print(tl3.types)
print()
tl4 = TfmdLists(fns, [_lbl, cat])
print(tl4.types)
tl5 = TfmdLists(fns, [_lbl, mult])
print(tl5.types)
tl6 = TfmdLists(fns, [cat, mult])
print(tl6.types)
print()
tl7 = TfmdLists(fns, [_lbl,cat,mult])
print(tl7.types)
print()
tl8 = TfmdLists(fns, [_lbl,cat,mult,mult])
print(tl8.types)

['dog_0.jpg', 'cat_0.jpg', 'cat_2.jpg', 'cat_1.jpg', 'dog_1.jpg']

[<class 'str'>, <class 'int'>]
[<class 'str'>, <class 'fastai.torch_core.TitledStr'>]
[<class 'str'>, <class 'str'>]

[<class 'str'>, <class 'fastai.torch_core.TitledStr'>, <class 'int'>]
[<class 'str'>, <class 'fastai.torch_core.TitledStr'>, <class 'fastai.torch_core.TitledStr'>]
[<class 'str'>, <class 'int'>, <class 'int'>]

[<class 'str'>, <class 'fastai.torch_core.TitledStr'>, <class 'int'>, <class 'int'>]

[<class 'str'>, <class 'fastai.torch_core.TitledStr'>, <class 'int'>, <class 'int'>, <class 'int'>]


In [ ]:
#hide
#Test input_types works on a Transform
cat = _Cat()
cat.input_types = (str, float)
tl = TfmdLists(fns, [_lbl,cat,mult])
print(tl.types)
test_eq(tl.infer_idx(2.0), 1)

[<class 'str'>, (<class 'str'>, <class 'float'>), <class 'int'>, <class 'int'>]


In [ ]:
#hide
#Test type annotations work on a function
def mult(x:(int,float)): return x*2
mult.order = 2
tl = TfmdLists(fns, [_lbl,_Cat(),mult])
print(tl.types)
test_eq(tl.infer_idx(2.0), 2)

[<class 'str'>, <class 'fastai.torch_core.TitledStr'>, (<class 'int'>, <class 'float'>), <class 'int'>]


## Datasets -

In [ ]:
print("Start below")

Start below


In [ ]:
'''
Signature: tuplify(o, use_list=False, match=None)
Source:   
def tuplify(o, use_list=False, match=None):
    "Make `o` a tuple"
    return tuple(L(o, use_list=use_list, match=match))
File:      /usr/local/lib/python3.6/dist-packages/fastcore/utils.py
Type:      function
'''
tuplify??

In [ ]:
#export
@docs
@delegates(TfmdLists)
class Datasets(FilteredBase):
    "A dataset that creates a tuple from each `tfms`, passed through `item_tfms`"
    def __init__(self, items=None, tfms=None, tls=None, n_inp=None, dl_type=None, **kwargs):
        super().__init__(dl_type=dl_type)
        # have a few TfmdLists, creates one TfmdLists for each list of tfms you pass in
        # so not one pipeline but n pipelines passed in, usually n is 2, a X (independent
        # variable) pipeline and a Y (dependent variable) pipeline. Same items but different
        # set of transforms.
        self.tls = L(tls if tls else [TfmdLists(items, t, **kwargs) for t in L(ifnone(tfms,[None]))])
        self.n_inp = ifnone(n_inp, max(1, len(self.tls)-1))

    def __getitem__(self, it):
        res = tuple([tl[it] for tl in self.tls])
        return res if is_indexer(it) else list(zip(*res))

    def __getattr__(self,k): return gather_attrs(self, k, 'tls')
    def __dir__(self): return super().__dir__() + gather_attr_names(self, 'tls')
    def __len__(self): return len(self.tls[0])
    def __iter__(self): return (self[i] for i in range(len(self)))
    def __repr__(self): return coll_repr(self)
    def decode(self, o, full=True): return tuple(tl.decode(o_, full=full) for o_,tl in zip(o,tuplify(self.tls, match=o)))
    def subset(self, i): return type(self)(tls=L(tl.subset(i) for tl in self.tls), n_inp=self.n_inp)
    def _new(self, items, *args, **kwargs): return super()._new(items, tfms=self.tfms, do_setup=False, **kwargs)
    def overlapping_splits(self): return self.tls[0].overlapping_splits()
    def new_empty(self): return type(self)(tls=[tl.new_empty() for tl in self.tls], n_inp=self.n_inp)
    @property
    def splits(self): return self.tls[0].splits
    @property
    def split_idx(self): return self.tls[0].tfms.split_idx
    @property
    def items(self): return self.tls[0].items
    @items.setter
    def items(self, v):
        for tl in self.tls: tl.items = v

    def show(self, o, ctx=None, **kwargs):
        for o_,tl in zip(o,self.tls): ctx = tl.show(o_, ctx=ctx, **kwargs)
        return ctx

    @contextmanager
    def set_split_idx(self, i):
        old_split_idx = self.split_idx
        for tl in self.tls: tl.tfms.split_idx = i
        try: yield self
        finally:
            for tl in self.tls: tl.tfms.split_idx = old_split_idx

    _docs=dict(
        decode="Compose `decode` of all `tuple_tfms` then all `tfms` on `i`",
        show="Show item `o` in `ctx`",
        dataloaders="Get a `DataLoaders`",
        overlapping_splits="All splits that are in more than one split",
        subset="New `Datasets` that only includes subset `i`",
        new_empty="Create a new empty version of the `self`, keeping only the transforms",
        set_split_idx="Contextmanager to use the same `Datasets` with another `split_idx`"
    )

A `Datasets` creates a tuple from `items` (typically input,target) by applying to them each list of `Transform` (or `Pipeline`) in `tfms`. Note that if `tfms` contains only one list of `tfms`, the items given by `Datasets` will be tuples of one element. 

`n_inp` is the number of elements in the tuples that should be considered part of the input and will default to 1 if `tfms` consists of one set of transforms, `len(tfms)-1` otherwise. In most cases, the number of elements in the tuples spit out by `Datasets` will be 2 (for input,target) but it can happen that there is 3 (Siamese networks or tabular data) in which case we need to be able to determine when the inputs end and the targets begin.

In [ ]:
items = [1,2,3,4]
dsets = Datasets(items, [[neg_tfm,int2f_tfm], [add(1)]])
t = dsets[0] # t = 1
# Both pipes applied to t which is 1
test_eq(t, (-1,2))
test_eq(dsets[0,1,2], [(-1,2),(-2,3),(-3,4)])
test_eq(dsets.n_inp, 1)
# add has no decode so input is output so 2 
# and -1 goes through int2f_tfm decode so float -1.0 then neg_tfm
dsets.decode(t)

(1.0, 2)

In [ ]:
class Norm(Transform):
    def encodes(self, o): return (o-self.m)/self.s
    def decodes(self, o): return (o*self.s)+self.m
    def setups(self, items):
        its = tensor(items).float()
        self.m,self.s = its.mean(),its.std()

In [ ]:
items = [1,2,3,4]
nrm = Norm()
dsets = Datasets(items, [[neg_tfm,int2f_tfm], [neg_tfm,nrm]])

print(nrm.m) # mean of [-1, -2, -3, -4] = tensor(-2.5)
print(nrm.s) # sd of [-1, -2, -3, -4] = tensor(1.29) 
print()

x,y = zip(*dsets)
print(x)
print(y)

# since y is normed its mean should be close to 0, sd close to 1
test_close(tensor(y).mean(), 0)
test_close(tensor(y).std(), 1)

# compare below to print value of x and print of nrm.m
test_eq(x, (-1,-2,-3,-4,))
test_eq(nrm.m, -2.5)

# notice that y does not have any ability to show itself, only x does
# also if thing itself here for eg output of encodes is TitledInt is capable
# of showing itself it will show it; only if that is NOT possible, it will
# do the decode until you find something you can show approach. so here
# it ends up showing TitledInt of -2 = '-2
test_stdout(lambda:show_at(dsets, 1), '-2')

# comparing attributes; also shows ways attributes can be accessed.
test_eq(dsets.m, nrm.m)
test_eq(dsets.norm.m, nrm.m)
test_eq(dsets.train.norm.m, nrm.m)

tensor(-2.5000)
tensor(1.2910)

(-1, -2, -3, -4)
(tensor(1.1619), tensor(0.3873), tensor(-0.3873), tensor(-1.1619))


In [ ]:
#hide
#Check filtering is properly applied
class B(Transform):
    def encodes(self, x)->None:  return int(x+1)
    def decodes(self, x):        return TitledInt(x-1)
add1 = B(split_idx=1)

# important to remember that splits are indices into the original items
dsets = Datasets(items, [neg_tfm, [neg_tfm,int2f_tfm,add1]], splits=[[3],[0,1,2]])

# dsets index [1] implies value 2, add1 tfm will not match due to split_idx being 1 
# hence dsets[1] is [-2, -2]
test_eq(dsets[1], [-2,-2])

# dsets.valid[1] is index 1 of dsets so -2 again but this index is match for add1
# hence -2, -1 result
test_eq(dsets.valid[1], [-2,-1])
test_eq(dsets.valid[[1,1]], [[-2,-1], [-2,-1]])

# dsets.train[0] is dsets index 3, which is value 4.
test_eq(dsets.train[0], [-4,-4])

In [ ]:
test_fns = ['dog_0.jpg','cat_0.jpg','cat_2.jpg','cat_1.jpg','kid_1.jpg']
tcat = _Cat()
# note due to order that even though below is [tcat,_lbl] it will execute
# _lbl, tcat as _lbl order is 0 as it is default, while _Cat() order is 1 (speced above)
dsets = Datasets(test_fns, [[tcat,_lbl]], splits=[[0,1,2], [3,4]])
test_eq(tcat.vocab, ['cat','dog'])
test_eq(dsets.train, [(1,),(0,),(0,)])
test_eq(dsets.valid[0], (0,))
test_stdout(lambda: show_at(dsets.train, 0), "dog")

In [ ]:
inp = [0,1,2,3,4]
dsets = Datasets(inp, tfms=[None])

test_eq(*dsets[2], 2)          # Retrieve one item (subset 0 is the default)
test_eq(dsets[1,2], [(1,),(2,)])    # Retrieve two items by index
mask = [True,False,False,True,False]
test_eq(dsets[mask], [(0,),(3,)])   # Retrieve two items by mask

In [ ]:
inp = pd.DataFrame(dict(a=[5,1,2,3,4]))
dsets = Datasets(inp, tfms=attrgetter('a')).subset(0)
test_eq(*dsets[2], 2)          # Retrieve one item (subset 0 is the default)
test_eq(dsets[1,2], [(1,),(2,)])    # Retrieve two items by index
mask = [True,False,False,True,False]
test_eq(dsets[mask], [(5,),(3,)])   # Retrieve two items by mask

Recall: n_inp is the number of elements in the tuples that should be considered part of the input and will default to 1 if tfms consists of one set of transforms, len(tfms)-1 otherwise

In [ ]:
#test n_inp
inp = [0,1,2,3,4]
dsets = Datasets(inp, tfms=[None])

# check default is 1
test_eq(dsets.n_inp, 1)

# if not specified check n_inp is len(tfms)-1
dsets = Datasets(inp, tfms=[[None],[None],[None]])
test_eq(dsets.n_inp, 2)

# if specified use n_imp and not len(tfms)-1
dsets = Datasets(inp, tfms=[[None],[None],[None]], n_inp=1)
test_eq(dsets.n_inp, 1)

In [ ]:
# splits can be tensors indices
dsets = Datasets(range(5), tfms=[None], splits=[tensor([0,2]), [1,3,4]])

test_eq(dsets.subset(0), [(0,),(2,)])
test_eq(dsets.train, [(0,),(2,)])       # Subset 0 is aliased to `train`
test_eq(dsets.subset(1), [(1,),(3,),(4,)])
test_eq(dsets.valid, [(1,),(3,),(4,)])     # Subset 1 is aliased to `valid`
test_eq(*dsets.valid[2], 4)
#assert '[(1,),(3,),(4,)]' in str(dsets) and '[(0,),(2,)]' in str(dsets)
dsets

(#5) [(0,),(1,),(2,),(3,),(4,)]

In [ ]:
# splits can be boolean masks (they don't have to cover all items, but must be disjoint)
# Seems to imply mask cannot be True in both (ensuring disjoint) 
# but can be False in both (not covering all items) - see index 3 False in both
splits = [[False,True,True,False,True], [True,False,False,False,False]]
dsets = Datasets(range(5), tfms=[None], splits=splits)

test_eq(dsets.train, [(1,),(2,),(4,)])
test_eq(dsets.valid, [(0,)])

In [ ]:
# apply transforms to all items
tfm = [[lambda x: x*2,lambda x: x+1]]
splits = [[1,2],[0,3,4]]
dsets = Datasets(range(5), tfm, splits=splits)
# so below is values at index 1 and 2 which are 1,2
# 1*2+1, 2*2+1 hence 3,5
test_eq(dsets.train,[(3,),(5,)])

# similar to above for val at indices 0,3,4
test_eq(dsets.valid,[(1,),(7,),(9,)])

# use mask
test_eq(dsets.train[False,True], [(5,)])

In [ ]:
# only transform subset 1
class _Tfm(Transform):
    split_idx=1
    def encodes(self, x): return x*2
    def decodes(self, x): return TitledStr(x//2)

In [ ]:
dsets = Datasets(range(5), [_Tfm()], splits=[[1,2],[0,3,4]])

# since split_idx is 1 for _Tfm() no match on training any index
# hence below returns orig values of dsets.train at indices 1,2
test_eq(dsets.train,[(1,),(2,)])

# since split_idx is matched values of dsets.valid at indices
# 0,3,4 which are values 0,3,4 are *2 and hence
test_eq(dsets.valid,[(0,),(6,),(8,)])

# mask on no returned values still works
test_eq(dsets.train[False,True], [(2,)])
dsets

(#5) [(0,),(1,),(2,),(3,),(4,)]

In [ ]:
#A context manager to change the split_idx and apply the validation transform on the training set
# So can use context mgr to change split_idx within context
# Notice that only within context mgr split_idx of ds.train is set to 1 not outside - so within context mgr
# since split_idx matches so dsets.train has
ds = dsets.train
with ds.set_split_idx(1):
    test_eq(ds,[(2,),(4,)])
test_eq(dsets.train,[(1,),(2,)])

In [ ]:
#hide
#Test Datasets pickles
dsrc1 = pickle.loads(pickle.dumps(dsets))
test_eq(dsets.train, dsrc1.train)
test_eq(dsets.valid, dsrc1.valid)

In [ ]:
dsets = Datasets(range(5), [_Tfm(),noop], splits=[[1,2],[0,3,4]])
# _Tfm() only if idx matches which it does not but noop on indices 1 and 2 returning tuples
test_eq(dsets.train,[(1,1),(2,2)])

# below seems like you passed in two pipelines one of _Tfm() and another noop ???
# see skr test below
test_eq(dsets.valid,[(0,0),(6,3),(8,4)])

In [ ]:
# skr test
dsets1 = Datasets(range(5), [[_Tfm()], [noop]], splits=[[1,2],[0,3,4]])
test_eq(dsets.train,[(1,1),(2,2)])
test_eq(dsets1.valid,[(0,0),(6,3),(8,4)])

In [ ]:
#class A(Transform): 
#    def encodes(self, x): return x 
#    def decodes(self, x): return TitledInt(x) 


In [ ]:
start = torch.arange(0,50)
tds = Datasets(start, [A()])
tdl = TfmdDL(tds, after_item=NegTfm(), bs=4)
print(L(tdl))
print(b)
b = tdl.one_batch()
# Since neg negates on encodes and also on encodes returns same value after decodes
test_eq(tdl.decode_batch(b), ((0,),(1,),(2,),(3,)))
test_stdout(tdl.show_batch, "0\n1\n2\n3")

(#13) [(tensor([ 0, -1, -2, -3]),),(tensor([-4, -5, -6, -7]),),(tensor([ -8,  -9, -10, -11]),),(tensor([-12, -13, -14, -15]),),(tensor([-16, -17, -18, -19]),),(tensor([-20, -21, -22, -23]),),(tensor([-24, -25, -26, -27]),),(tensor([-28, -29, -30, -31]),),(tensor([-32, -33, -34, -35]),),(tensor([-36, -37, -38, -39]),)...]
(tensor([ 0, -1, -2, -3]),)


In [ ]:
# only transform subset 1
class _Tfm(Transform):
    split_idx=1
    def encodes(self, x): return x*2

dsets = Datasets(range(8), [None], splits=[[1,2,5,7],[0,3,4,6]])

In [ ]:
# only transform subset 1
class _Tfm(Transform):
    split_idx=1
    def encodes(self, x): return x*2

dsets = Datasets(range(8), [None], splits=[[1,2,5,7],[0,3,4,6]])
dls = dsets.dataloaders(bs=4, after_batch=_Tfm(), shuffle_train=False, device=torch.device('cpu'))
test_eq(dls.train, [(tensor([1,2,5, 7]),)])
test_eq(dls.valid, [(tensor([0,6,8,12]),)])
test_eq(dls.n_inp, 1)

### Methods

In [ ]:
items = [1,2,3,4]
dsets = Datasets(items, [[neg_tfm,int2f_tfm]])

In [ ]:
#hide_input
_dsrc = Datasets([1,2])
show_doc(_dsrc.dataloaders, name="Datasets.dataloaders")

NameError: ignored

In [ ]:
show_doc(Datasets.decode)

<h4 id="Datasets.decode" class="doc_header"><code>Datasets.decode</code><a href="__main__.py#L20" class="source_link" style="float:right">[source]</a></h4>

> <code>Datasets.decode</code>(**`o`**, **`full`**=*`True`*)

Compose `decode` of all `tuple_tfms` then all `tfms` on `i`

In [ ]:
test_eq(*dsets[0], -1)
test_eq(*dsets.decode((-1,)), 1)

In [ ]:
show_doc(Datasets.show)

<h4 id="Datasets.show" class="doc_header"><code>Datasets.show</code><a href="__main__.py#L35" class="source_link" style="float:right">[source]</a></h4>

> <code>Datasets.show</code>(**`o`**, **`ctx`**=*`None`*, **\*\*`kwargs`**)

Show item `o` in `ctx`

In [ ]:
test_stdout(lambda:dsets.show(dsets[1]), '-2')

In [ ]:
show_doc(Datasets.new_empty)

<h4 id="Datasets.new_empty" class="doc_header"><code>Datasets.new_empty</code><a href="__main__.py#L24" class="source_link" style="float:right">[source]</a></h4>

> <code>Datasets.new_empty</code>()

Create a new empty version of the `self`, keeping only the transforms

In [ ]:
items = [1,2,3,4]
nrm = Norm()
dsets = Datasets(items, [[neg_tfm,int2f_tfm], [neg_tfm]])
empty = dsets.new_empty()
test_eq(empty.items, [])

In [ ]:
#hide
#test it works for dataframes too
df = pd.DataFrame({'a':[1,2,3,4,5], 'b':[6,7,8,9,10]})
dsets = Datasets(df, [[attrgetter('a')], [attrgetter('b')]])
empty = dsets.new_empty()

## Add test set for inference

In [ ]:
# only transform subset 1
#class _Tfm(Transform):
#    split_idx=1
#    def encodes(self, x): return x*2

In [ ]:
# only transform subset 0
class _Tfm1(Transform):
    split_idx=0
    def encodes(self, x): return x*3

dsets = Datasets(range(8), [[_Tfm(),_Tfm1()]], splits=[[1,2,5,7],[0,3,4,6]])

# execute appropriate transforms from pipeline where split_idx matches
# so _Tmf1() for training set since split_idx=0 matches training set idx which is 0
test_eq(dsets.train, [(3,),(6,),(15,),(21,)])

# so _Tfm() for valid set since split_idx=1 matches valid set idx which is 1
test_eq(dsets.valid, [(0,),(6,),(8,),(12,)])

In [ ]:
#export
def test_set(dsets, test_items, rm_tfms=None, with_labels=False):
    "Create a test set from `test_items` using validation transforms of `dsets`"
    if isinstance(dsets, Datasets):
        tls = dsets.tls if with_labels else dsets.tls[:dsets.n_inp]
        test_tls = [tl._new(test_items, split_idx=1) for tl in tls]
        if rm_tfms is None: rm_tfms = [tl.infer_idx(get_first(test_items)) for tl in test_tls]
        else:               rm_tfms = tuplify(rm_tfms, match=test_tls)
        for i,j in enumerate(rm_tfms): test_tls[i].tfms.fs = test_tls[i].tfms.fs[j:]
        return Datasets(tls=test_tls)
    elif isinstance(dsets, TfmdLists):
        test_tl = dsets._new(test_items, split_idx=1)
        if rm_tfms is None: rm_tfms = dsets.infer_idx(get_first(test_items))
        test_tl.tfms.fs = test_tl.tfms.fs[rm_tfms:]
        return test_tl
    else: raise Exception(f"This method requires using the fastai library to assemble your data. Expected a `Datasets` or a `TfmdLists` but got {dsets.__class__.__name__}")

In [ ]:
class _Tfm1(Transform):
    split_idx=0
    def encodes(self, x): return x*3

dsets = Datasets(range(8), [[_Tfm(),_Tfm1()]], splits=[[1,2,5,7],[0,3,4,6]])
test_eq(dsets.train, [(3,),(6,),(15,),(21,)])
test_eq(dsets.valid, [(0,),(6,),(8,),(12,)])

#Tranform of the validation set are applied and you have to provide test_items
tst = test_set(dsets, [9,10,11])
test_eq(tst, [(18,),(20,),(22,)])

In [ ]:
#hide
#Test with different types
tfm = _Tfm1()
tfm.split_idx,tfm.order = None,2
dsets = Datasets(['dog', 'cat', 'cat', 'dog'], [[_Cat(),tfm]])

#With strings
# 3*1 for dog, 3*0 for cat
test_eq(test_set(dsets, ['dog', 'cat', 'cat']), [(3,), (0,), (0,)])
#With ints
test_eq(test_set(dsets, [1,2]), [(3,), (6,)])

In [ ]:
#hide
#Test with various input lengths
dsets = Datasets(range(8), [[_Tfm(),_Tfm1()],[_Tfm(),_Tfm1()],[_Tfm(),_Tfm1()]], splits=[[1,2,5,7],[0,3,4,6]])
tst = test_set(dsets, [1,2,3])
# Here you get a tuple coz without n_inp defined it is now len(tfms)-1 = 2 hence a tuple with 2 values
test_eq(tst, [(2,2),(4,4),(6,6)])

dsets = Datasets(range(8), [[_Tfm(),_Tfm1()],[_Tfm(),_Tfm1()],[_Tfm(),_Tfm1()]], splits=[[1,2,5,7],[0,3,4,6]], n_inp=1)
tst = test_set(dsets, [1,2,3])
# Here n_inp is explicitly one so you only get a single valued tuple as output
test_eq(tst, [(2,),(4,),(6,)])

Note: rm_tfms is defined with test_set. See above.

In [ ]:
#hide
#Test with rm_tfms
dsets = Datasets(range(8), [[_Tfm(),_Tfm()]], splits=[[1,2,5,7],[0,3,4,6]])
tst = test_set(dsets, [1,2,3])
test_eq(tst, [(4,),(8,),(12,)])

dsets = Datasets(range(8), [[_Tfm(),_Tfm()]], splits=[[1,2,5,7],[0,3,4,6]])

# seeing code for rm_tfms in test_set seems remove tfms starting at index specified
# in rm_tfms if multiple pipeline specified in tuple
tst = test_set(dsets, [1,2,3], rm_tfms=1)
test_eq(tst, [(2,),(4,),(6,)])

dsets = Datasets(range(8), [[_Tfm(),_Tfm()], [_Tfm(),_Tfm()]], splits=[[1,2,5,7],[0,3,4,6]], n_inp=2)

# use of rm_tfms tuples says remove tfms starting at index 1 in 1st pipeline and 0 imply no changes to second pipeline.
tst = test_set(dsets, [1,2,3], rm_tfms=(1,0))
test_eq(tst, [(2,4),(4,8),(6,12)])

In [ ]:
#export
@delegates(TfmdDL.__init__)
@patch
def test_dl(self:DataLoaders, test_items, rm_type_tfms=None, with_labels=False, **kwargs):
    "Create a test dataloader from `test_items` using validation transforms of `dls`"
    test_ds = test_set(self.valid_ds, test_items, rm_tfms=rm_type_tfms, with_labels=with_labels
                      ) if isinstance(self.valid_ds, (Datasets, TfmdLists)) else test_items
    return self.valid.new(test_ds, **kwargs)

In [ ]:
dsets = Datasets(range(8), [[_Tfm(),_Tfm1()]], splits=[[1,2,5,7],[0,3,4,6]])
dls = dsets.dataloaders(bs=4, device=torch.device('cpu'))

In [ ]:
dsets = Datasets(range(8), [[_Tfm(),_Tfm1()]], splits=[[1,2,5,7],[0,3,4,6]])
dls = dsets.dataloaders(bs=4, device=torch.device('cpu'))
tst_dl = dls.test_dl([2,3,4,5])
test_eq(tst_dl._n_inp, 1)
test_eq(list(tst_dl), [(tensor([ 4,  6,  8, 10]),)])
#Test you can change transforms
tst_dl = dls.test_dl([2,3,4,5], after_item=add1)
test_eq(list(tst_dl), [(tensor([ 5,  7,  9, 11]),)])

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_torch_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_data.load.ipynb.
Converted 03_data.core.ipynb.
Converted 04_data.external.ipynb.
Converted 05_data.transforms.ipynb.
Converted 06_data.block.ipynb.
Converted 07_vision.core.ipynb.
Converted 08_vision.data.ipynb.
Converted 09_vision.augment.ipynb.
Converted 09b_vision.utils.ipynb.
Converted 09c_vision.widgets.ipynb.
Converted 10_tutorial.pets.ipynb.
Converted 11_vision.models.xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_callback.core.ipynb.
Converted 13a_learner.ipynb.
Converted 13b_metrics.ipynb.
Converted 14_callback.schedule.ipynb.
Converted 14a_callback.data.ipynb.
Converted 15_callback.hook.ipynb.
Converted 15a_vision.models.unet.ipynb.
Converted 16_callback.progress.ipynb.
Converted 17_callback.tracker.ipynb.
Converted 18_callback.fp16.ipynb.
Converted 18a_callback.training.ipynb.
Converted 19_callback.mixup.ipynb.
Converted 20_interpret.ipynb.
Converted 20a_distributed.ipynb.
Converted 21_vision.l